# Multi-Agent Solution: Connected Agent for Inventory and Policy Q&A

This notebook demonstrates how to build a multi-agent solution using Azure AI Foundry, where a main agent delegates tasks to a specialized connected agent:
- **Knowledge Base Agent**: Answers questions about company policies (e.g., return policy) and inventory using a file-based knowledge base.

The main agent routes user queries to the connected agent, following the [Connected Agents pattern](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/connected-agents?pivots=portal).

**Step 1**: Install NuGet packages

To get started with Azure AI Foundry, you need to install the required NuGet packages:
- `Microsoft.SemanticKernel`
- `Microsoft.Extensions.AI.AzureAIInference`
- `Microsoft.SemanticKernel.Agents.Core`
- `DotNetEnv`
- `Microsoft.SemanticKernel.Agents.AzureAI`

In [36]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7"
#r "nuget: Microsoft.SemanticKernel.Agents.Core"
#r "nuget: DotNetEnv, 3.1.0"
#r "nuget: Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview"

Installed Packages DotNetEnv, 3.1.0 Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7 Microsoft.SemanticKernel, 1.56.0 Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview Microsoft.SemanticKernel.Agents.Core, 1.56.0

**Step 2**: Read environment variables

In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.

In [37]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\ai-foundry\../..\.env


**Step 3**: Instantiate the Project client

Create an instance of the `PersistentAgentsClient` using the endpoint URL and the Azure AI Foundry project.

In [38]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Agents.AzureAI;
using Azure.Identity;
using Azure.AI.Agents.Persistent;

var projectEndpoint = Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_ENDPOINT");
#pragma warning disable SKEXP0110
var client = AzureAIAgent.CreateAgentsClient(projectEndpoint, new DefaultAzureCredential());
#pragma warning restore SKEXP0110

**Step 4a**: Create the Knowledge Base Agent (Company Policy Q&A)

This agent will answer questions about company policies, especially the return policy, using the provided documentation.

In [39]:
var modelId = Environment.GetEnvironmentVariable("MODEL");
var policyAgentName = "ContosoPolicyAgent";
var policyAgentInstructions = @"
    You are a helpful assistant for Contoso Bikes.
    Answer questions about company policies, especially the return policy, using the provided documentation.
    If a question is not covered by the documentation, politely inform the user.
";
var policyFilePath = Path.Combine(Environment.CurrentDirectory, "../../resources/contoso-bikes.pdf");
var policyFileStream = File.OpenRead(policyFilePath);
PersistentAgentFileInfo policyFileInfo = await client.Files.UploadFileAsync(policyFileStream, PersistentAgentFilePurpose.Agents, "contoso-bikes.pdf");
PersistentAgentsVectorStore policyFileStore = await client.VectorStores.CreateVectorStoreAsync([policyFileInfo.Id]);
var policyAgentDef = await client.Administration.CreateAgentAsync(
    modelId,
    name: policyAgentName,
    instructions: policyAgentInstructions,
    tools: [new FileSearchToolDefinition()],
    toolResources: new()
    {
        FileSearch = new()
        {
            VectorStoreIds = { policyFileStore.Id },
        }
    });


#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var policyAgent=  new AzureAIAgent(policyAgentDef,client);

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 4b**: Create the Inventory Management Agent

This agent will answer questions about bike inventory using the Contoso Bike Store API.

In [40]:
// Inventory Management Agent (Action Tool)
var inventoryAgentName = "ContosoInventoryAgent";
var inventoryAgentInstructions = @"
You are a knowledgeable and friendly assistant for the Contoso Bike Store.
Your job is to help customers find information about bikes, check inventory using the Contoso Bike Store API.
Always provide clear, concise, and accurate responses. If you need more details from the user, ask clarifying questions.
Use the API to retrieve up-to-date information.
";
var inventoryApiPath = Path.Combine(Environment.CurrentDirectory, "../../resources/bike-inventory-api-swagger.json");
var inventoryApiData = File.ReadAllBytes(inventoryApiPath);
OpenApiAnonymousAuthDetails inventoryOaiAuth = new();
OpenApiToolDefinition inventoryOpenApiTool = new(
    name: "ContosoBikeStoreAPI",
    description: "The API provides information about the bikes in the catalog for the Contoso Bike Store.",
    spec: BinaryData.FromBytes(inventoryApiData),
    openApiAuthentication: inventoryOaiAuth,
    defaultParams: ["format"]
);
var inventoryAgentDef = await client.Administration.CreateAgentAsync(
    modelId,
    name: inventoryAgentName,
    instructions: inventoryAgentInstructions,
    tools: [inventoryOpenApiTool],
    toolResources: null);


#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var inventoryAgent=  new AzureAIAgent(inventoryAgentDef,client);

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.      


**Step 5**: Create the Main Orchestrator Agent

The main agent will delegate to the policy Q&A agent using ConnectedAgentToolDefinition.

In [41]:
var mainAgentName = "ContosoMainAgent";
var mainAgentInstructions = @"
You are the main assistant for Contoso Bike Store. 
- If the user asks about company policies (such as returns, warranties, or store rules), delegate to the policy assistant.
- If the user asks about bike inventory, availability, or product details, delegate to the inventory assistant.
- If the question is unrelated to these topics, politely inform the user that you can only assist with company policies and inventory questions.
";

ConnectedAgentToolDefinition policyAgentConnection = new(new ConnectedAgentDetails(policyAgent.Id, policyAgent.Name, "Answers questions about company policies, especially the return policy."));
ConnectedAgentToolDefinition inventoryAgentConnection = new(new ConnectedAgentDetails(inventoryAgent.Id, inventoryAgent.Name, "Answers questions about the inventory of bikes in the Contoso Bike Store."));

var mainAgentDef = await client.Administration.CreateAgentAsync(
    modelId,
    name: mainAgentName,
    instructions: mainAgentInstructions,
    tools: [policyAgentConnection, inventoryAgentConnection],
    toolResources: null);

**Step 6**: Create the Main Agent Instance and Thread

In [42]:
#pragma warning disable SKEXP0110
var mainAgent = new AzureAIAgent(mainAgentDef, client);
var thread = new AzureAIAgentThread(client);
#pragma warning restore SKEXP0110

**Step 7**: Interact with the Multi-Agent System

You can now interact with the main agent, which will route your queries to the connected agent.

In [43]:
#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var invokeAgentAsync = async (AzureAIAgentThread agentThread, string input) =>
{
    ChatMessageContent message = new(AuthorRole.User, input);
    await foreach (ChatMessageContent response in mainAgent.InvokeAsync(message, agentThread))
    {
        Console.WriteLine($"{response.Role}: {response.Content}");
    }
};

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

// Example 1: Ask about return policy
await invokeAgentAsync(thread, "What is the return policy at Contoso Bikes?");

// // Example 2: Ask about company policy
// await invokeAgentAsync(thread, "Are there any items that cannot be returned?");

assistant: Contoso Bikes' return policy includes the following:

1. **Eligibility**: Items must be unused, in original packaging, with proof of purchase. Exclusions include custom bikes, gift cards, and clearance items.
2. **Return Period**: Most items can be returned within 30 days of delivery for a refund or exchange.
3. **Process**: Contact customer service for an RMA number and shipping instructions, securely package the item, and use a trackable shipping method.
4. **Refunds/Exchanges**: Refunds are credited to the original payment method within seven business days after inspection. Exchanges depend on availability.
5. **Shipping Costs**: Return shipping costs are covered by the customer unless it’s due to a defect or error from Contoso (prepaid label provided). International returns may incur additional fees.
6. **Damaged/Defective Items**: Report within seven days of delivery for a replacement or refund at no extra cost .

Let me know if you'd like further assistance!


Now ask about the available bikes at a specific location.


In [44]:
await invokeAgentAsync(thread, "Do you have Contoso Mountain X1 at the Seattle store?");

assistant: The **Contoso Mountain X1** is available at the **Seattle - Northgate store**, with **8 units in stock**, priced at **$1199.99**. It’s a durable mountain bike built for tough terrains. Let me know if you’d like to place an order or need more details!


**Step 8**: Clean Up Resources

Delete the thread and all agents to clean up resources.

In [ ]:
await thread.DeleteAsync();
await client.Administration.DeleteAgentAsync(policyAgent.Id);
await client.Administration.DeleteAgentAsync(inventoryAgent.Id);
await client.Administration.DeleteAgentAsync(mainAgent.Id);